# Understanding Memory in LLMs

In the previous Notebook 03, we successfully explored how OpenAI models can enhance the results from Azure Cognitive Search. [Bing Chat](http://chat.bing.com/) is a search engine with a GPT-4 model that utilizes the content of search results to provide context and deliver accurate responses to queries.

However, we have yet to discover how to engage in a conversation with the LLM. With Bing Chat, this is possible, as the LLM can understand and reference the previous responses.

There is a common misconception that GPT models have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

The aim of this Notebook is to demonstrate how we can "provide memory" to the LLM by utilizing prompts and context.

In [1]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferMemory
from openai.error import OpenAIError
from langchain.docstore.document import Document

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from app.utils import (
    get_search_results,
    order_search_results,
    model_tokens_limit,
    num_tokens_from_docs,
    embed_docs,
    search_docs,
    get_answer,
)

from app.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT, COMBINE_CHAT_PROMPT

# Don't mess with this unless you really know what you are doing
AZURE_SEARCH_API_VERSION = '2021-04-30-Preview'
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"

# Change these below with your own services credentials

AZURE_SEARCH_ENDPOINT = "https://azure-cog-search-xxullfkgszplc.search.windows.net/"
AZURE_SEARCH_KEY = "DX3OIwAPkXDq9dv4sYdn1qqfp9nNbWx9v0DAUG4rQTAzSeA4hmxI" # Make sure is the MANAGEMENT KEY no the query key
AZURE_OPENAI_ENDPOINT = "https://openainlp1.openai.azure.com/"
AZURE_OPENAI_API_KEY = "9e64f10580c44989b8fabe9b23336344"

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["AZURE_SEARCH_KEY"] = AZURE_SEARCH_KEY
os.environ["AZURE_SEARCH_ENDPOINT"] = AZURE_SEARCH_ENDPOINT
os.environ["OPENAI_API_TYPE"] = "azure"

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [3]:
QUESTION = "Tell me some use cases for reinforcement learning?"
FOLLOW_UP_QUESTION = "Can you rephrase what you just said?"

In [4]:
# Define model
MODEL = "gpt-35-turbo"
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0.5)

In [5]:
# We create a very simple prompt template, just the question as is:
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [6]:
# Let's see what the GPT model responds
response = chain.run(QUESTION)
printmd(response)

1. Robotics: Reinforcement learning can be used to train robots to perform complex tasks such as grasping, manipulation, and navigation.

2. Game Playing: Reinforcement learning can be used to train agents to play games such as chess, Go, and poker.

3. Autonomous Vehicles: Reinforcement learning can be used to train self-driving cars to make decisions and navigate through traffic.

4. Advertising: Reinforcement learning can be used to optimize ad placement and targeting to maximize revenue.

5. Finance: Reinforcement learning can be used to optimize trading strategies and portfolio management.

6. Healthcare: Reinforcement learning can be used to optimize treatment plans for patients with chronic diseases.

7. Energy Management: Reinforcement learning can be used to optimize energy usage in buildings and power grids.

8. Recommendation Systems: Reinforcement learning can be used to personalize recommendations for users based on their preferences and behavior.

9. Supply Chain Management: Reinforcement learning can be used to optimize inventory management and logistics operations.

10. Education: Reinforcement learning can be used to personalize learning experiences for students based on their progress and performance.

In [7]:
#Now let's ask a follow up question
chain.run(FOLLOW_UP_QUESTION)

'I apologize, but as an AI language model, I cannot recall what I said previously. Could you please provide me with the sentence or phrase that you want me to rephrase?'

As you can see, it doesn't remember what it just responded. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [8]:
hist_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
                {history}
                Human: {question}
                AI:
            """
    )
chain = LLMChain(llm=llm, prompt=hist_prompt)

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response)

In [10]:
chain.run({"history":Conversation_history, "question": FOLLOW_UP_QUESTION})

'Sure, some examples of how reinforcement learning can be applied include training robots for complex tasks, game playing, autonomous vehicles, optimizing advertising and finance strategies, healthcare treatment plans, energy management, recommendation systems, supply chain management, and personalized education.'

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

In order to not duplicate code, we have put many of the code used in Notebook 3 into functions. These functions are in the app/utils.py and app/prompts.py files This way we can use these functios in the app that we will build later.

In [11]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

agg_search_results = get_search_results(QUESTION, indexes)
ordered_results = order_search_results(agg_search_results, reranker_threshold=1)

In [12]:
docs = []
for key,value in ordered_results.items():
    for page in value["chunks"]:
        docs.append(Document(page_content=page, metadata={"source": value["location"]}))

# Calculate number of tokens of our docs
tokens_limit = model_tokens_limit(MODEL)

if(len(docs)>0):
    num_tokens = num_tokens_from_docs(docs)
    print("Custom token limit for", MODEL, ":", tokens_limit)
    print("Combined docs tokens count:",num_tokens)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")


Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 52495


In [13]:
%%time
if num_tokens > tokens_limit:
    index = embed_docs(docs)
    top_docs = search_docs(index,QUESTION)
    
    # Now we need to recalculate the tokens count of the top results from similarity vector search
    # in order to select the chain type: stuff or map_reduce
    
    num_tokens = num_tokens_from_docs(top_docs)   
    print("Token count after similarity search:", num_tokens)
    chain_type = "map_reduce" if num_tokens > tokens_limit else "stuff"
    
else:
    # if total tokens is less than our limit, we don't need to vectorize and do similarity search
    top_docs = docs
    chain_type = "stuff"
    
print("Chain Type selected:", chain_type)

Number of chunks: 48
Token count after similarity search: 4110
Chain Type selected: map_reduce
CPU times: user 247 ms, sys: 13.1 ms, total: 260 ms
Wall time: 5.36 s


In [14]:
# Get the answer
response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type)
response['output_text']

'Reinforcement learning has been used in applications such as board games, opponent modeling in commercial games, Sokoban, verification of game design and playing strategies, position evaluation in the game of Othello, and ensemble pruning. It has also been applied to routing. Reinforcement learning research focuses on designing algorithms for an agent interacting with an environment to optimize a long-term return. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

And if we ask the follow up question:

In [15]:
response = get_answer(docs=top_docs,  query=FOLLOW_UP_QUESTION, language="English",deployment=MODEL, chain_type=chain_type)
response['output_text']

'In the given portion of the document, there is no relevant text to rephrase. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf'

Until now we just have the same as the prior Notebook 03: results from Azure Search enhanced by OpenAI model, with no memory

**Now let's add memory to it:**

Reference: https://python.langchain.com/en/latest/modules/memory/examples/adding_memory_chain_multiple_inputs.html

In [16]:
# memory object, which is neccessary to track the inputs/outputs and hold a conversation.
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question")

response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

'Reinforcement learning has been used in a variety of applications, including board games, opponent modeling and commercial games, enhancing single-agent search methods, verifying game design and playing strategies, multi-agent reinforcement learning, ensemble pruning, and learning to play the game of chess. It has also been used in routing applications. While the given portions of the documents do not provide a direct answer to your question, they suggest that reinforcement learning can be used to optimize long-term returns by searching for optimal behaviors in a class of behaviors. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

In [17]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

'I apologize for any confusion. What I meant was that reinforcement learning has been used in various applications such as board games, opponent modeling, commercial games, and routing applications to optimize long-term returns by searching for optimal behaviors in a class of behaviors. The documents I provided suggest that value estimators have been developed to estimate the value of using one policy over another, resulting in more data-efficient algorithms. Does that help clarify my previous response?\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

In [18]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

"I'm sorry, but I don't have enough information to answer your question. Is there anything else I can help you with?\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf"

Let's check our memory to see that it's keeping the conversation

In [19]:
memory.buffer

"Human: Tell me some use cases for reinforcement learning?\nAI: Reinforcement learning has been used in a variety of applications, including board games, opponent modeling and commercial games, enhancing single-agent search methods, verifying game design and playing strategies, multi-agent reinforcement learning, ensemble pruning, and learning to play the game of chess. It has also been used in routing applications. While the given portions of the documents do not provide a direct answer to your question, they suggest that reinforcement learning can be used to optimize long-term returns by searching for optimal behaviors in a class of behaviors. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0611/0611163v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf\nHuman: Can you rephrase what you just said?\nAI: I apologize for any confusion. What I meant was that reinforcement lea

Bad pipe message: %s [b"\x07\xc7n5?\xaa\xda\xb7\t\xbf\xbe\x9f\xde\xf5\x93\x91\x14{ \xe3l\xeab\xb6C\x14\xbe\xed\x8d\x9e\x1ci\xb2pt\xbe'93\xdf\xc7U]1\xd2\xaf\xad*c[\xbf\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08", b'\x05\x08\x06']
Bad pipe message: %s [b'\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x88 l\x10\xe4\x13zG\xfd\x1e\xee\xd7\xb6J\xcd\x8d\xb6\xc1\xf0\x95\x1bF']
Bad pipe message: %s [b'l*\xb3\xccH\xc3PT\x81y\\\xa8\xaf\x98\x0c\xb3H\xcc \xee\x84\xcd\x8c&2+0v\xbeB\xfa+\xda\x98\xa8\xf5\x87\xf5\x12\xa3CTbdF\xf2\xc2$Q\x85\xcd\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x0

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, it struggles to respond to prompts like: Hello, Thank you, Bye, What's your name, What's the weather and any other task that is not search in the knowledge base



# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

But, does this solve all the possible scenarios that a virtual assistant will require?  **What about if the answer to the Smart Search Engine is not related to text, but instead requires to look into tabular data?** The next notebook 04 explains and solves the tabular problem and the concept of Agents